# Demonstrate ADES Execution for OGC Application Packages
## This notebook runs through some example API calls to the ADES (Application, Deployment Execution Service) component of the EODH Platform

In [1]:
!pip install urllib3

Defaulting to user installation because normal site-packages is not writeable


In [4]:
import json
import time
import urllib3
http = urllib3.PoolManager(cert_reqs='CERT_NONE')
urllib3.disable_warnings() ## to avoid SSL warnings

In [5]:
## Define text colour for later output
class bcolors:
    OKBLUE = '\033[94m'
    OKGREEN = '\033[92m'
    WARNING = '\033[93m'
    ENDC = '\033[0m'

In [ ]:
## Place your workspace-scoped API token here
api_token = "your-token"
workspace = "workspace" # must align with the workspace-scoped token used above
auth_dict = {"Authorization": f"Bearer {api_token}"}

## Below are some example API requests you can make to the ADES component
Feel free to run these examples and change the inputs by specifying the application packages, process name and process inputs.

As an example we provide three EOEPCA-developed OGC Application Package to demonstrate the successful execution using the ADES deployment:
- [convert-url](https://github.com/EOEPCA/convert/blob/main/convert-url-app.cwl) - take an image specified by a URL and resize it by a given scale percentage
- [convert-stac](https://github.com/EOEPCA/convert/blob/main/convert-stac-app.cwl) - take an image specified by a stac item and resize it by a given scale percentage
- [water-bodies](https://github.com/EOEPCA/deployment-guide/blob/main/deploy/samples/requests/processing/water-bodies-app.cwl) - takes STAC items, area of interest, epsg definition and set of bands and identifies water bodies based on NDWI and Otsu threshold

This application is specified by configuring the below variable

In [7]:
process_to_be_run = "convert-url"

In [ ]:
# Update these variables as required to identify the running ades instance and specify workspace name
# If the workspace does not yet exect, it will be created by the ades automatically
wr_endpoint = "prod.eodatahub.org.uk/api/catalogue/stac/catalogs/user/catalogs"

# Automated configuration of CWL script location, process name and inputs
if process_to_be_run == "convert-url":
    process_name = "convert-url"
    cwl_location = "https://raw.githubusercontent.com/EO-DataHub/eodhp-ades-demonstration/refs/heads/main/convert-url-app.cwl"
    inputs_dict = {"inputs": {
                    "fn": "resize",
                    "url":  "https://eoepca.org/media_portal/images/logo6_med.original.png",
                    "size": "50%"
                    }
                  }
elif process_to_be_run == "convert-stac":
    process_name = "convert-stac"
    cwl_location = "https://raw.githubusercontent.com/EO-DataHub/eodhp-ades-demonstration/refs/heads/main/convert-stac-app.cwl"
    inputs_dict = {"inputs": {
                    "fn": "resize",
                    "stac":  "https://raw.githubusercontent.com/EOEPCA/convert/main/stac/eoepca-logo.json",
                    "size": "50%"
                    }
                  }

elif process_to_be_run == "water-bodies":
    process_name = "water-bodies"
    cwl_location = "https://raw.githubusercontent.com/EO-DataHub/eodhp-ades-demonstration/refs/heads/main/water-bodies-app.cwl"
    inputs_dict = {"inputs": {
                    "stac_items": [
                        "https://prod.eodatahub.org.uk/catalogue-data/element84-data/collections/sentinel-2-c1-l2a/items/S2B_T42MVU_20240319T054135_L2A.json"
                    ],
                    "aoi": "68.09, -6.42, 69.09, -5.43",
                    "epsg": "EPSG:4326",
                    "bands": [
                        "green",
                        "nir"
                    ]
                    }
                  }

### List processes

In [ ]:
url = f"https://{wr_endpoint}/{workspace}/processes"
headers = {"Accept": "application/json"}
headers.update(auth_dict)
response = http.request('GET', url, headers=headers)
json.loads(response.data)

{'processes': [{'id': 'display',
   'title': 'Print Cheetah templates as HTML',
   'description': 'Print Cheetah templates as HTML.',
   'mutable': False,
   'version': '2.0.0',
   'jobControlOptions': ['sync-execute', 'async-execute', 'dismiss'],
   'outputTransmission': ['value', 'reference'],
   'links': [{'rel': 'self',
     'type': 'application/json',
     'title': 'Process Description',
     'href': 'https://dev.eodatahub.org.uk/ades/test_cluster_3/ogc-api/processes/display'}]},
  {'id': 'echo',
   'title': 'Echo input',
   'description': 'Simply echo the value provided as input',
   'mutable': False,
   'version': '2.0.0',
   'metadata': [{'title': 'Demo'}],
   'jobControlOptions': ['sync-execute', 'async-execute', 'dismiss'],
   'outputTransmission': ['value', 'reference'],
   'links': [{'rel': 'self',
     'type': 'application/json',
     'title': 'Process Description',
     'href': 'https://dev.eodatahub.org.uk/ades/test_cluster_3/ogc-api/processes/echo'}]}],
 'links': [{'rel

### Undeploy/Delete process

In [ ]:
## Here a 204 response means the process was remove successfully, no other content is returned
url = f"https://{wr_endpoint}/{workspace}/processes/{process_name}"
headers = {"Accept": "application/json"}
headers.update(auth_dict)
params = {}
response = http.request('DELETE', url, headers=headers)
response.status

403

### Deploy processes

In [ ]:
url = f"https://{wr_endpoint}/{workspace}/processes"
headers = {"Accept": "application/json", "Content-Type": "application/json"}
headers.update(auth_dict)
params = {"executionUnit": {
            "href": f"{cwl_location}",
            "type": "application/cwl"
            }
         }
response = http.request('POST', url, headers=headers, body=json.dumps(params))
deployStatus = response.headers['Location']
json.loads(response.data)

{'id': 'convert-url',
 'title': 'convert url app',
 'description': 'Convert URL',
 'mutable': True,
 'version': '0.1.2',
 'outputTransmission': ['value', 'reference'],
 'jobControlOptions': ['async-execute', 'dismiss'],
 'links': [{'rel': 'http://www.opengis.net/def/rel/ogc/1.0/execute',
   'type': 'application/json',
   'title': 'Execute End Point',
   'href': 'https://dev.eodatahub.org.uk/ades/test_cluster_3/ogc-api/processes/convert-url/execution'}]}

### Get deploy status

In [10]:
url = f"{deployStatus}"
headers = {"Accept": "application/json"}
headers.update(auth_dict)
params = {}
response = http.request('GET', url, headers=headers)
json.loads(response.data)

{'id': 'convert-url',
 'title': 'convert url app',
 'description': 'Convert URL',
 'mutable': True,
 'version': '0.1.2',
 'outputTransmission': ['value', 'reference'],
 'jobControlOptions': ['async-execute', 'dismiss'],
 'links': [{'rel': 'http://www.opengis.net/def/rel/ogc/1.0/execute',
   'type': 'application/json',
   'title': 'Execute End Point',
   'href': 'https://dev.eodatahub.org.uk/ades/test_cluster_3/ogc-api/processes/convert-url/execution'}],
 'inputs': {'fn': {'title': 'the operation to perform',
   'description': 'the operation to perform',
   'schema': {'type': 'string'}},
  'size': {'title': 'the percentage for a resize operation',
   'description': 'the percentage for a resize operation',
   'schema': {'type': 'string'}},
  'url': {'title': 'the image to convert',
   'description': 'the image to convert',
   'schema': {'type': 'string'}}},
 'outputs': {'converted_image': {'title': 'converted_image',
   'description': 'None',
   'extended-schema': {'oneOf': [{'allOf': [{

### Get process details

In [ ]:
url = f"https://{wr_endpoint}/{workspace}/processes/{process_name}"
headers = {"Accept": "application/json"}
headers.update(auth_dict)
params = {}
response = http.request('GET', url, headers=headers)
json.loads(response.data)

{'id': 'convert-url',
 'title': 'convert url app',
 'description': 'Convert URL',
 'mutable': True,
 'version': '0.1.2',
 'outputTransmission': ['value', 'reference'],
 'jobControlOptions': ['async-execute', 'dismiss'],
 'links': [{'rel': 'http://www.opengis.net/def/rel/ogc/1.0/execute',
   'type': 'application/json',
   'title': 'Execute End Point',
   'href': 'https://dev.eodatahub.org.uk/ades/test_cluster_3/ogc-api/processes/convert-url/execution'}],
 'inputs': {'fn': {'title': 'the operation to perform',
   'description': 'the operation to perform',
   'schema': {'type': 'string'}},
  'size': {'title': 'the percentage for a resize operation',
   'description': 'the percentage for a resize operation',
   'schema': {'type': 'string'}},
  'url': {'title': 'the image to convert',
   'description': 'the image to convert',
   'schema': {'type': 'string'}}},
 'outputs': {'converted_image': {'title': 'converted_image',
   'description': 'None',
   'extended-schema': {'oneOf': [{'allOf': [{

### Execute process

In [ ]:
url = f"https://{wr_endpoint}/{workspace}/processes/{process_name}/execution"
headers = {"Accept": "application/json", "Content-Type": "application/json", "Prefer": "respond-async"}
headers.update(auth_dict)
params = {**inputs_dict}
print(json.dumps(params))
time.sleep(5)
response = http.request('POST', url, headers=headers, body=json.dumps(params))
executeStatus = response.headers['Location']
json.loads(response.data)

{"inputs": {"fn": "resize", "url": "https://eoepca.org/media_portal/images/logo6_med.original.png", "size": "50%"}}


{'jobID': 'e555a7a8-e6d4-11ee-a53b-229b4c2f5d9c',
 'type': 'process',
 'processID': 'convert-url',
 'created': '2024-03-20T16:14:15.347Z',
 'started': '2024-03-20T16:14:15.347Z',
 'updated': '2024-03-20T16:14:15.347Z',
 'status': 'running',
 'message': 'ZOO-Kernel accepted to run your service!',
 'links': [{'title': 'Status location',
   'rel': 'status',
   'type': 'application/json',
   'href': 'https://dev.eodatahub.org.uk/ades/test_cluster_3/ogc-api/jobs/e555a7a8-e6d4-11ee-a53b-229b4c2f5d9c'}]}

### Get execute status
See the following section to continually poll this function instead to determine once complete

In [13]:
url = f"{executeStatus}"
headers = {"Accept": "application/json"}
headers.update(auth_dict)
params = {}
time.sleep(5)
response = http.request('GET', url, headers=headers)
json.loads(response.data)

{'progress': 10,
 'jobID': 'e555a7a8-e6d4-11ee-a53b-229b4c2f5d9c',
 'type': 'process',
 'processID': 'convert-url',
 'created': '2024-03-20T16:14:15.347Z',
 'started': '2024-03-20T16:14:15.347Z',
 'updated': '2024-03-20T16:14:17.045Z',
 'status': 'running',
 'message': 'workflow wrapped, creating processing environment',
 'links': [{'title': 'Status location',
   'rel': 'status',
   'type': 'application/json',
   'href': 'https://dev.eodatahub.org.uk/ades/test_cluster_3/ogc-api/jobs/e555a7a8-e6d4-11ee-a53b-229b4c2f5d9c'}]}

### Get execute status (continuous polling)
Run this cell to keep polling the ExecuteStatus endpoint to determine when the process has finished running and also see it's final status: *SUCCESS* or *FAILED*

In [14]:
url = f"{executeStatus}"
headers = {"Accept": "application/json"}
headers.update(auth_dict)
params = {}
response = http.request('GET', url, headers=headers)
data = json.loads(response.data)
status = data['status']
message = data['message']
print("Status is " + bcolors.OKBLUE + status.upper() + bcolors.ENDC)
print("Message is " + "\033[1m" + message + "\033[0m", end="")
old_message = message
old_status = status
while status == "running":
    time.sleep(2)
    response = http.request('GET', url, headers=headers)
    data = json.loads(response.data)
    status = data['status']
    message = data['message']
    if status != old_status:
        print("\n")
        print("Status is " + bcolors.OKBLUE + status.upper() + bcolors.ENDC)
        print("Message is " + "\033[1m" + message + "\033[0m", end="")
    elif message != old_message:
        print(".")
        print("Message is " + "\033[1m" + message + "\033[0m", end="")
    else:
        print(".", end="")
    old_message = message
    old_status = status

if status == "successful":
    print("\n")
    print(bcolors.OKGREEN + "SUCCESS" + bcolors.ENDC)

if status == "failed":
    print(bcolors.WARNING + "FAILED" + bcolors.ENDC)

Status is RUNNING
Message is workflow wrapped, creating processing environment..........
Message is processing environment created, preparing execution......
Message is execution submitted............
Message is delivering outputs, logs and usage report.....
Message is execution successful

Status is SUCCESSFUL
Message is ZOO-Kernel successfully run your service!

SUCCESS


### Get processing results

In [18]:
## Note, this will return a 500 response when no output is produced
url = f"{executeStatus}/results"
headers = {"Accept": "application/json"}
headers.update(auth_dict)
params = {}
response = http.request('GET', url, headers=headers)
json.loads(response.data)

{}

### List jobs

In [ ]:
url = f"https://{wr_endpoint}/{workspace}/jobs"
headers = {"Accept": "application/json"}
headers.update(auth_dict)
params = {}
response = http.request('GET', url, headers=headers)
json.loads(response.data)

{'jobs': [{'jobID': 'e555a7a8-e6d4-11ee-a53b-229b4c2f5d9c',
   'type': 'process',
   'processID': 'convert-url',
   'created': '2024-03-20T16:14:15.347Z',
   'started': '2024-03-20T16:14:15.347Z',
   'finished': '2024-03-20T16:15:48.826Z',
   'updated': '2024-03-20T16:15:48.311Z',
   'status': 'successful',
   'message': 'ZOO-Kernel successfully run your service!',
   'links': [{'title': 'Status location',
     'rel': 'status',
     'type': 'application/json',
     'href': 'https://dev.eodatahub.org.uk/ades/test_cluster_3/ogc-api/jobs/e555a7a8-e6d4-11ee-a53b-229b4c2f5d9c'},
    {'title': 'Result location',
     'rel': 'http://www.opengis.net/def/rel/ogc/1.0/results',
     'type': 'application/json',
     'href': 'https://dev.eodatahub.org.uk/ades/test_cluster_3/ogc-api/jobs/e555a7a8-e6d4-11ee-a53b-229b4c2f5d9c/results'},
    {'href': 'https://dev.eodatahub.org.uk/ades/temp/convert-url-e555a7a8-e6d4-11ee-a53b-229b4c2f5d9c/convert.log',
     'title': 'Tool log convert.log',
     'rel': '

### Delete a Running Job

In [ ]:
job_id = "your-job-id"
url = f"https://{wr_endpoint}/{workspace}/jobs/{job_id}"
headers = {"Accept": "application/json"}
headers.update(auth_dict)
## Uncomment the following lines if you wish to delete the running Job as specified in `job_id` above
# response = http.request('DELETE', url, headers=headers)
# response.status

### Undeploy/Delete process

In [ ]:
## Here a 204 response means the process was remove successfully, no other content is returned
url = f"https://{wr_endpoint}/{workspace}/processes/{process_name}"
headers = {"Accept": "application/json"}
headers.update(auth_dict)
params = {}
response = http.request('DELETE', url, headers=headers)
response.status


204